In [50]:
import pandas as pd
import numpy as np
import os
import pycocotools.mask as mutils
import base64
import numpy as np
from pycocotools import mask as coco_mask_d
import typing as t
import zlib
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm

DATA_DIR = '/mnt/chicm/data/open-images/segmentation'

In [51]:
def get_top_classes(start_index, end_index, class_file='top_classes_level1.csv'):
    df = pd.read_csv(os.path.join(DATA_DIR, class_file))
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi


In [52]:
level1_classes = get_top_classes(0, 275)[0]
parent_classes = get_top_classes(0, 25, 'top_classes_parent.csv')[0]

seg_classes = set(level1_classes) | set(parent_classes)
len(seg_classes)

300

In [54]:
len(level1_classes)

275

In [55]:
df = pd.read_csv('/mnt/chicm/open-images-vrd/vrd/det/final_add0.001.zip')

In [56]:
#df = pd.read_csv('seg_lb5026_bbox.csv')

In [57]:
df.PredictionString = df.PredictionString.fillna('')

In [58]:
df.head()

,ImageId,PredictionString
0,00000b4dcff7f799,/m/07yv9 0.1014 0.1469 0.1145 0.9360 0.8608 /m...
1,00001a21632de752,/m/07yv9 0.1103 0.0052 0.0577 0.8222 0.9935 /m...
2,0000d67245642c5f,/m/07yv9 0.0457 0.0370 0.0690 0.9413 0.9923 /m...
3,0001244aa8ed3099,/m/07yv9 0.1433 0.0512 0.1449 0.8007 0.7719 /m...
4,000172d1dd1adce0,/m/07yv9 0.1140 0.2272 0.0299 0.8484 0.9221 /m...


In [59]:
def get_dets_by_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return []
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    dets, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 6 == 0:
            assert '/' in det[0]
            if det[0] in level1_classes:
                dets.append(det)
            det = []
    return sorted(dets, key=lambda x: float(x[1]), reverse=True)[:2000]

In [60]:
#sorted(ens_det, key=lambda x: x[2], reverse=True)[:MAX_NUM]

In [61]:
df['dets'] = df.PredictionString.map(lambda x: get_dets_by_pred_str(x))
df.head()

,ImageId,PredictionString,dets
0,00000b4dcff7f799,/m/07yv9 0.1014 0.1469 0.1145 0.9360 0.8608 /m...,"[[/m/07r04, 0.8837, 0.0093, 0.1279, 0.7010, 0...."
1,00001a21632de752,/m/07yv9 0.1103 0.0052 0.0577 0.8222 0.9935 /m...,"[[/m/03bt1vf, 0.7607, 0.5409, 0.5901, 0.8157, ..."
2,0000d67245642c5f,/m/07yv9 0.0457 0.0370 0.0690 0.9413 0.9923 /m...,"[[/m/079cl, 0.2479, 0.2416, 0.0406, 0.9919, 0...."
3,0001244aa8ed3099,/m/07yv9 0.1433 0.0512 0.1449 0.8007 0.7719 /m...,"[[/m/0388q, 0.1431, 0.8231, 0.5525, 0.8828, 0...."
4,000172d1dd1adce0,/m/07yv9 0.1140 0.2272 0.0299 0.8484 0.9221 /m...,"[[/m/09rvcxw, 0.2363, 0.4385, 0.0337, 0.7900, ..."


In [62]:
# total
df.dets.map(lambda x: len(x)).values.sum()

43677184

In [58]:
# total
df.dets.map(lambda x: len(x)).values.sum()

28818241

In [65]:
TEST_IMG_DIR = '/mnt/chicm/data/open-images/test'

def get_fn(img_id):
    return TEST_IMG_DIR + '/' + img_id + '.jpg'

df['ImageWidth'] = df.ImageId.map(lambda x: get_image_size(get_fn(x))[0])
df['ImageHeight'] = df.ImageId.map(lambda x: get_image_size(get_fn(x))[1])
df.head()

,ImageId,PredictionString,dets,ImageWidth,ImageHeight
0,00000b4dcff7f799,/m/07yv9 0.1014 0.1469 0.1145 0.9360 0.8608 /m...,"[[/m/07r04, 0.8837, 0.0093, 0.1279, 0.7010, 0....",1024,683
1,00001a21632de752,/m/07yv9 0.1103 0.0052 0.0577 0.8222 0.9935 /m...,"[[/m/03bt1vf, 0.7607, 0.5409, 0.5901, 0.8157, ...",1024,681
2,0000d67245642c5f,/m/07yv9 0.0457 0.0370 0.0690 0.9413 0.9923 /m...,"[[/m/079cl, 0.2479, 0.2416, 0.0406, 0.9919, 0....",1024,683
3,0001244aa8ed3099,/m/07yv9 0.1433 0.0512 0.1449 0.8007 0.7719 /m...,"[[/m/0388q, 0.1431, 0.8231, 0.5525, 0.8828, 0....",1024,683
4,000172d1dd1adce0,/m/07yv9 0.1140 0.2272 0.0299 0.8484 0.9221 /m...,"[[/m/09rvcxw, 0.2363, 0.4385, 0.0337, 0.7900, ...",1024,682


In [66]:
df.rename(columns={'ImageId': 'ImageID'}, inplace=True)
df.head()

,ImageID,PredictionString,dets,ImageWidth,ImageHeight
0,00000b4dcff7f799,/m/07yv9 0.1014 0.1469 0.1145 0.9360 0.8608 /m...,"[[/m/07r04, 0.8837, 0.0093, 0.1279, 0.7010, 0....",1024,683
1,00001a21632de752,/m/07yv9 0.1103 0.0052 0.0577 0.8222 0.9935 /m...,"[[/m/03bt1vf, 0.7607, 0.5409, 0.5901, 0.8157, ...",1024,681
2,0000d67245642c5f,/m/07yv9 0.0457 0.0370 0.0690 0.9413 0.9923 /m...,"[[/m/079cl, 0.2479, 0.2416, 0.0406, 0.9919, 0....",1024,683
3,0001244aa8ed3099,/m/07yv9 0.1433 0.0512 0.1449 0.8007 0.7719 /m...,"[[/m/0388q, 0.1431, 0.8231, 0.5525, 0.8828, 0....",1024,683
4,000172d1dd1adce0,/m/07yv9 0.1140 0.2272 0.0299 0.8484 0.9221 /m...,"[[/m/09rvcxw, 0.2363, 0.4385, 0.0337, 0.7900, ...",1024,682


In [17]:
df.to_csv('od627_convert_275_rpn_top2000.csv', columns=['ImageID', 'ImageWidth', 'ImageHeight', 'dets'], index=False)

In [8]:
df = pd.read_csv('od627_convert_275_rpn_top2000.csv', converters={'dets': eval})
df.head()

,ImageID,ImageWidth,ImageHeight,dets
0,00000b4dcff7f799,1024,683,"[[/m/07r04, 0.8837, 0.0093, 0.1279, 0.7010, 0...."
1,00001a21632de752,1024,681,"[[/m/03bt1vf, 0.7607, 0.5409, 0.5901, 0.8157, ..."
2,0000d67245642c5f,1024,683,"[[/m/079cl, 0.2479, 0.2416, 0.0406, 0.9919, 0...."
3,0001244aa8ed3099,1024,683,"[[/m/0388q, 0.1431, 0.8231, 0.5525, 0.8828, 0...."
4,000172d1dd1adce0,1024,682,"[[/m/09rvcxw, 0.2363, 0.4385, 0.0337, 0.7900, ..."


In [67]:
df_test = pd.read_csv(os.path.join(DATA_DIR, 'sample_empty_submission.csv')) #.iloc[:1000]
df_test.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,NaN
1,52a76f3fe21185bf,1024,683,NaN
2,65e9da7c04c8e4cd,1024,682,NaN
3,6d10fa21d93bd6cc,1024,682,NaN
4,0841dee573a875ca,1024,683,NaN


In [68]:
df = df.set_index('ImageID')
df = df.reindex(index=df_test['ImageID'])
df = df.reset_index()
df.head()

,ImageID,PredictionString,dets,ImageWidth,ImageHeight
0,80155d58d0ee19bd,/m/07yv9 0.1719 0.2581 0.2637 0.7300 0.8100 /m...,"[[/m/01kb5b, 0.6820, 0.2582, 0.2638, 0.7184, 0...",1024,683
1,52a76f3fe21185bf,/m/07yv9 0.0042 0.0469 0.0638 0.9991 0.9943 /m...,"[[/m/0bjyj5, 0.6689, 0.6231, 0.0653, 0.9808, 0...",1024,683
2,65e9da7c04c8e4cd,/m/07yv9 0.0180 0.0000 0.0396 0.9014 0.9985 /m...,"[[/m/0fszt, 0.0350, 0.0040, 0.0848, 0.9904, 0....",1024,682
3,6d10fa21d93bd6cc,/m/07yv9 0.0130 0.2894 0.2498 0.6292 0.4203 /m...,"[[/m/04yx4, 0.8521, 0.5103, 0.0276, 0.8860, 0....",1024,682
4,0841dee573a875ca,/m/07yv9 0.2048 0.3297 0.8043 0.5641 0.9975 /m...,"[[/m/0342h, 0.6931, 0.4476, 0.4630, 0.7242, 0....",1024,683


In [69]:
df.dets.values[0]

[['/m/01kb5b', '0.6820', '0.2582', '0.2638', '0.7184', '0.8074'],
 ['/m/01k6s3', '0.5169', '0.2763', '0.2722', '0.7170', '0.8217'],
 ['/m/07dd4', '0.3075', '0.2477', '0.2710', '0.7330', '0.8053'],
 ['/m/025dyy', '0.2648', '0.2599', '0.2668', '0.7188', '0.8018'],
 ['/m/0dv5r', '0.2457', '0.2563', '0.2679', '0.7215', '0.8109'],
 ['/m/025dyy', '0.2452', '0.3055', '0.2821', '0.7166', '0.7788'],
 ['/m/020lf', '0.2101', '0.2705', '0.2644', '0.7172', '0.7720'],
 ['/m/0gjkl', '0.1718', '0.2571', '0.2637', '0.7208', '0.7850'],
 ['/m/03m3vtv', '0.1652', '0.2632', '0.2684', '0.7168', '0.8071'],
 ['/m/083wq', '0.1547', '0.3846', '0.6551', '0.4426', '0.7529'],
 ['/m/083wq', '0.1487', '0.2618', '0.2931', '0.3925', '0.6609'],
 ['/m/012n7d', '0.1467', '0.2666', '0.3001', '0.7246', '0.8053'],
 ['/m/01s55n', '0.1393', '0.2646', '0.2686', '0.7152', '0.8160'],
 ['/m/0h8lkj8', '0.1390', '0.2632', '0.2501', '0.7270', '0.8228'],
 ['/m/04h8sr', '0.1297', '0.2647', '0.2683', '0.7234', '0.7995'],
 ['/m/083wq', 

In [13]:
type(df.iloc[0].dets[0][1])

str

In [70]:
proposals = []

for dets, h, w in tqdm(zip(df.dets.values, df.ImageHeight.values, df.ImageWidth.values), total=df.shape[0]):
    img_proposal = []
    for det in dets:
        xmin, ymin, xmax, ymax = int(w*float(det[2])), int(h*float(det[3])), int(w*float(det[4])), int(h*float(det[5]))
        conf = float(det[1])
        img_proposal.append(np.array([xmin, ymin, xmax, ymax, conf]).astype(np.float32))
    proposals.append(np.stack(img_proposal))
    

100%|██████████| 99999/99999 [17:09<00:00, 97.16it/s] 


In [71]:
p = np.array(proposals)

In [72]:
p[1].shape

(531, 5)

In [73]:
np.save('proposals_627.npy', p)

In [45]:
p_dev = np.array(proposals[:1000])

In [46]:
np.save('proposals_dev_1000.npy', p_dev)

In [47]:
tmp = np.load('proposals.npy')

In [49]:
tmp[0].shape

(101, 5)

In [10]:
df.shape

(99999, 4)

In [11]:
df_test.shape

(99999, 4)

In [20]:
df_tmp = df_tmp.set_index('ImageID')

In [21]:
df_tmp.head()

,ImageWidth,ImageHeight,dets
ImageID,,,
00000b4dcff7f799,1024,683,"[[/m/07r04, 0.8837, 0.0093, 0.1279, 0.7010, 0...."
00001a21632de752,1024,681,"[[/m/03bt1vf, 0.7607, 0.5409, 0.5901, 0.8157, ..."
0000d67245642c5f,1024,683,"[[/m/079cl, 0.2479, 0.2416, 0.0406, 0.9919, 0...."
0001244aa8ed3099,1024,683,"[[/m/0388q, 0.1431, 0.8231, 0.5525, 0.8828, 0...."
000172d1dd1adce0,1024,682,"[[/m/09rvcxw, 0.2363, 0.4385, 0.0337, 0.7900, ..."


In [22]:
df_tmp.iloc['00001a21632de752'].dets[0]

TypeError: Cannot index by location index with a non-integer key

In [64]:
import struct
import imghdr

def get_image_size(fname):
    '''Determine the image type of fhandle and return its size.
    from draco'''
    with open(fname, 'rb') as fhandle:
        head = fhandle.read(24)
        if len(head) != 24:
            raise AssertionError('imghead len != 24')
        if imghdr.what(fname) == 'png':
            check = struct.unpack('>i', head[4:8])[0]
            if check != 0x0d0a1a0a:
                raise AssertionError('png check failed')
            width, height = struct.unpack('>ii', head[16:24])
        elif imghdr.what(fname) == 'gif':
            width, height = struct.unpack('<HH', head[6:10])
        elif imghdr.what(fname) == 'jpeg':
            try:
                fhandle.seek(0) # Read 0xff next
                size = 2
                ftype = 0
                while not 0xc0 <= ftype <= 0xcf:
                    fhandle.seek(size, 1)
                    byte = fhandle.read(1)
                    while ord(byte) == 0xff:
                        byte = fhandle.read(1)
                    ftype = ord(byte)
                    size = struct.unpack('>H', fhandle.read(2))[0] - 2
                # We are at a SOFn block
                fhandle.seek(1, 1)  # Skip `precision' byte.
                height, width = struct.unpack('>HH', fhandle.read(4))
            except Exception: #IGNORE:W0703
                raise
        else:
            print(fname, imghdr.what(fname))
            #raise AssertionError('file format not supported')
            img = cv2.imread(fname)
            print(img.shape)
            height, width, _ = img.shape

        return width, height
